In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\project\\Text Summerization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass( frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from CutYourText.constants import *
from CutYourText.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__( self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml( config_filepath )
        self.params = read_yaml( params_filepath )
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config( self ):
        config = self.config.data_transformation
        
        create_directories( [config.root_dir] )
        
        data_transformation_config = DataTransformationConfig( 
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
        return data_transformation_config

In [8]:
import os
from CutYourText.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset

e:\project\Text Summerization\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-16 21:01:34,201: INFO: config: PyTorch version 2.4.0+cu121 available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        # Tokenize the input dialogue
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        
        # Tokenize the summary as target
        target_encodings = self.tokenizer(text_target=example_batch['summary'], max_length=128, truncation=True)
        
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_dialogsum = load_dataset('csv', data_files=self.config.data_path)
        dataset_dialogsum_pt = dataset_dialogsum.map(self.convert_examples_to_features, batched=True)
        dataset_dialogsum_pt.save_to_disk(os.path.join(self.config.root_dir, "dialogsum_dataset"))


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-08-16 21:03:04,813: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-16 21:03:04,822: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-16 21:03:04,823: INFO: common: created directory at: artifacts]
[2024-08-16 21:03:04,824: INFO: common: created directory at: artifacts/data_transformation]


e:\project\Text Summerization\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
e:\project\Text Summerization\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarn